### Librerias utilizadas

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from area import area
import ipydatetime

### Histograma de tiempos de viaje para un año dado

In [2]:
def histograma_por_año(año):
    
    df = pd.read_csv("Data/OD_{}.csv".format(año))
    duration = df["duration_sec"]
    plt.hist(duration, bins=20 ,histtype='bar', edgecolor='k')
    plt.xlabel('Duracion del viaje(segundos)')
    plt.ylabel('Numero de viajes')
    plt.title('Histograma de tiempos de viaje año {}'.format(año))
    
    return plt.show()

In [3]:
interact(histograma_por_año, año=[2014,2015,2016,2017])

interactive(children=(Dropdown(description='año', options=(2014, 2015, 2016, 2017), value=2014), Output()), _d…

<function __main__.histograma_por_año>

### Listado del Top N de estaciones más utilizadas para un año dado. Dividirlo en:
- Estaciones de salida
- Estaciones de llegada
- En general

In [64]:
def listado_top_estaciones_año(año,N):
    
    df = pd.read_csv("Data/OD_{}.csv".format(año))
    
    start_st = pd.DataFrame({'station_code': df['start_station_code']})
    end_st = pd.DataFrame({'station_code': df['end_station_code']})
    general_st = start_st.append(end_st)
    
    start_st = start_st.groupby('station_code').size().reset_index(name='counts')
    start_st = start_st.sort_values(by=['counts'],ascending=False)
    start_st = start_st.head(int(N))
    start_st.index = range(1, int(N)+1)
    
    end_st = end_st.groupby('station_code').size().reset_index(name='counts')
    end_st = end_st.sort_values(by=['counts'],ascending=False)
    end_st = end_st.head(int(N))
    end_st.index = range(1, int(N)+1)
    
    general_st = general_st.groupby('station_code').size().reset_index(name='counts')
    general_st = general_st.sort_values(by=['counts'],ascending=False)
    general_st = general_st.head(int(N))
    general_st.index = range(1, int(N)+1)
    
    informe = pd.concat([start_st,end_st,general_st],axis=1)
    informe.columns = ["start_station_code","start_count","end_station_code","end_count","general_station_code","general_count"]
    informe.to_csv("listado_top{}_estaciones_{}.csv".format(int(N),año),index=False)
    
    return informe

In [65]:
interact(listado_top_estaciones_año, año=[2014,2015,2016,2017],N=(0.0,200.0,1))

interactive(children=(Dropdown(description='año', options=(2014, 2015, 2016, 2017), value=2014), FloatSlider(v…

<function __main__.listado_top_estaciones_año>

### Listado del Top N de viajes más comunes para un año dado. Donde un viaje se define por su estación de salida y de llegada

In [68]:
def listado_top_viajes_año(año,N):
    
    df = pd.read_csv("Data/OD_{}.csv".format(año))
    
    viajes = pd.DataFrame({'start_station_code': df['start_station_code'],
                      'end_station_code': df['end_station_code']})
    
    viajes["viaje"] = viajes["start_station_code"].astype(str) + '-' + viajes["end_station_code"].astype(str)
    viajes = viajes.groupby('viaje').size().reset_index(name='counts')
    viajes = viajes.sort_values(by=['counts'],ascending=False)
    viajes = viajes.head(int(N))
    viajes.index = range(1, int(N)+1)
    viajes.to_csv("listado_top{}_viajes_{}.csv".format(int(N),año),index=False)
    
    return viajes

In [69]:
interact(listado_top_viajes_año, año=[2014,2015,2016,2017],N=(0.0,200.0,1))

interactive(children=(Dropdown(description='año', options=(2014, 2015, 2016, 2017), value=2014), FloatSlider(v…

<function __main__.listado_top_viajes_año>

### Identificación de horas punta para un año determinado sin tener en cuenta el día. Es decir, si es día de semana, fin de semana, festivo o temporada del año.

In [70]:
def histograma_horas_por_año(año):
    
    df = pd.read_csv("Data/OD_{}.csv".format(año))
    hours = pd.to_datetime(df["start_date"]).dt.hour
    plt.hist(hours, bins=24 ,histtype='bar', edgecolor='k')
    plt.xlabel('Hora del viage')
    plt.xticks(range(0,24)) 
    plt.xlabel('Hora')
    plt.ylabel('Numero de viajes')
    plt.title('Histograma de horas puntas de viaje año {}'.format(año))
    
    return plt.show()

In [71]:
interact(histograma_horas_por_año, año=[2014,2015,2016,2017])

interactive(children=(Dropdown(description='año', options=(2014, 2015, 2016, 2017), value=2014), Output()), _d…

<function __main__.histograma_horas_por_año>

### Comparación de utilización del sistema entre dos años cualesquiera. La utilización del sistema se puede medir como:
- Cantidad de viajes totales
- Tiempo total de utilización del sistema
- Cantidad de viajes por estaciones/bicicletas disponibles


In [74]:
def comparacion_sistema_año(año1,año2,medida):
    
    df1 = pd.read_csv("Data/OD_{}.csv".format(año1))
    df2 = pd.read_csv("Data/OD_{}.csv".format(año2))
    
    if medida=="viajes totales":
        
        plt.bar([str(año1),str(año2)],[len(df1),len(df2)])
        plt.xlabel('Año')
        plt.ylabel('Numero de viajes')
        plt.title('Comparacion Nº viajes entre el {} y el {}'.format(año1,año2))
        return plt.show()
    
    if medida=="tiempo de uso":
        
        uso1 = str(pd.Timedelta(df1["duration_sec"].sum(), unit ='s'))
        uso2 = str(pd.Timedelta(df2["duration_sec"].sum(), unit ='s'))
        
        usoTotal = pd.DataFrame({str(año1):uso1,
                                 str(año2):uso2},index=[0])
        
        return usoTotal
        
    else:
        start_st1 = pd.DataFrame({'station_code': df1['start_station_code']})
        end_st1 = pd.DataFrame({'station_code': df1['end_station_code']})
        general_st1 = start_st1.append(end_st1)
        
        start_st2 = pd.DataFrame({'station_code': df2['start_station_code']})
        end_st2 = pd.DataFrame({'station_code': df2['end_station_code']})
        general_st2 = start_st2.append(end_st2)
        
        st1 = general_st1.groupby('station_code').size().reset_index(name='counts')
        st1.index = range(1, len(st1)+1)
        
        st2 = general_st2.groupby('station_code').size().reset_index(name='counts')
        st2.index = range(1, len(st2)+1)

        comp_est = pd.concat([st1,st2],axis=1)
        comp_est.columns = ["station_code_"+str(año1),"count_"+str(año1),"station_code_"+str(año2),"count_"+str(año2)]
        comp_est.to_csv("comparacion_{}_{}_{}.csv".format(año1,año2,medida),index=False)
        
        return comp_est

In [75]:
interact(comparacion_sistema_año, año1=[2014,2015,2016,2017],año2=[2014,2015,2016,2017],
         medida=["viajes totales","tiempo de uso", "cantidad de viajes por estacion"])

interactive(children=(Dropdown(description='año1', options=(2014, 2015, 2016, 2017), value=2014), Dropdown(des…

<function __main__.comparacion_sistema_año>

### Capacidad instalada total (suma de la capacidad total de cada estación)

In [187]:
stations = pd.read_json("Data/stations.json")
capacity = pd.json_normalize(stations["stations"])
print("\nCapacidad total : {}".format(capacity['ba'].sum()))


Capacidad total : 4925


### Cambio en la capacidad instalada entre dos años puntuales

In [247]:
def comparacion_capacidad_año(año1,año2):
    
    df1 = pd.read_csv("Data/Stations_{}.csv".format(año1))
    df2 = pd.read_csv("Data/Stations_{}.csv".format(año2))
    
    stations = pd.read_json("Data/stations.json")
    capacity = pd.json_normalize(stations["stations"])
    
    sum1 = capacity[capacity.n.astype(int).isin(df1["code"])]
    sum2 = capacity[capacity.n.astype(int).isin(df2["code"])]
    
    change = sum2['ba'].sum() - sum1['ba'].sum()
    
    print("El cambio de capacidad entre los años {} y {} ha sido de : {} unidades".format(año1,año2,change))

In [248]:
interact(comparacion_capacidad_año, año1=[2014,2015,2016,2017],año2=[2014,2015,2016,2017])

interactive(children=(Dropdown(description='año1', options=(2014, 2015, 2016, 2017), value=2014), Dropdown(des…

<function __main__.comparacion_capacidad_año>

### Ampliación de la cobertura de la red entre dos años puntuales. La misma se puede medir como el área total que generan las estaciones.


In [327]:
def ampliacion_red_año(año1,año2):
    
    df1 = pd.read_csv("Data/Stations_{}.csv".format(año1))
    df2 = pd.read_csv("Data/Stations_{}.csv".format(año2))
    
    coordinates1 = []
    for i in range(len(df1)):
        coordinates1.append([df1["latitude"][i],df1["longitude"][i]])
        
    coordinates2 = []
    for i in range(len(df2)):
        coordinates2.append([df2["latitude"][i],df2["longitude"][i]])
    
    coordinates1.sort(key = lambda x: (-x[0], x[1]))
    coordinates2.sort(key = lambda x: (-x[0], x[1]))
    
    obj1 = {'type':'Polygon','coordinates':[coordinates1]}
    obj2 = {'type':'Polygon','coordinates':[coordinates2]}
    
    area1 = area(obj1)
    area2 = area(obj2)
    
    print("La diferencia entre la cobertura de la red entre el {} y el {} es: {} m".format(año1,año2,area2-area1))

In [328]:
interact(ampliacion_red_año, año1=[2014,2015,2016,2017],año2=[2014,2015,2016,2017])

interactive(children=(Dropdown(description='año1', options=(2014, 2015, 2016, 2017), value=2014), Dropdown(des…

<function __main__.ampliacion_red_año>

### Comparación de densidad de la red para un par de años puntuales. La densidad de la red se mide como el área que abarcan todas las estaciones,dividida la cantidad de estaciones.

In [335]:
def densidad_red_año(año1,año2):
    
    df1 = pd.read_csv("Data/Stations_{}.csv".format(año1))
    df2 = pd.read_csv("Data/Stations_{}.csv".format(año2))
    
    coordinates1 = []
    for i in range(len(df1)):
        coordinates1.append([df1["latitude"][i],df1["longitude"][i]])
        
    coordinates2 = []
    for i in range(len(df2)):
        coordinates2.append([df2["latitude"][i],df2["longitude"][i]])
    
    coordinates1.sort(key = lambda x: (-x[0], x[1]))
    coordinates2.sort(key = lambda x: (-x[0], x[1]))
    
    obj1 = {'type':'Polygon','coordinates':[coordinates1]}
    obj2 = {'type':'Polygon','coordinates':[coordinates2]}
    
    densidad1 = area(obj1) / len(df1)
    densidad2 = area(obj2) / len(df2)
    
    print("La diferencia entre la densidad de la red entre el {} y el {} es: {}".format(año1,año2,densidad2-densidad1))

In [336]:
interact(densidad_red_año, año1=[2014,2015,2016,2017],año2=[2014,2015,2016,2017])

interactive(children=(Dropdown(description='año1', options=(2014, 2015, 2016, 2017), value=2014), Dropdown(des…

<function __main__.densidad_red_año>

### Velocidad promedio de los ciclistas para un año determinado

In [425]:
from math import sin, cos, sqrt, atan2, radians 

def velocidad_año(año):
    
    df = pd.read_csv("Data/OD_{}.csv".format(año))
    stations = pd.read_json("Data/stations.json")
    stations = pd.json_normalize(stations["stations"])
    stations.n = stations.n.astype(int)
    
    start = pd.DataFrame({'n': df["start_station_code"]})
    result_start = pd.merge(start,stations,on='n',how="left")

    end = pd.DataFrame({'n': df["end_station_code"]})
    result_end = pd.merge(end,stations,on='n',how="left")

    combine = pd.concat([result_start[["n","id","la","lo"]],result_end[["n","id","la","lo"]],df["duration_sec"]],axis=1)
    combine.columns = ["start_station_code","id","start_station_la","start_station_lo",
                       "end_station_code","end_station_id","end_station_la","end_station_lo","duration_sec"]
    
    R = 6373.0 

    lat1 = combine["start_station_la"].map(radians)
    lon1 = combine["start_station_lo"].map(radians)
    lat2 = combine["end_station_la"].map(radians)
    lon2 = combine["end_station_lo"].map(radians)


    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (dlat/2).map(sin)**2 + lat1.map(cos) * lat2.map(cos) * (dlon/2).map(sin)**2 
    c = 2 * a.map(sqrt).map(asin) 

    distance = R * c

    combine["speed"] = distance / (combine["duration_sec"]/3600)
    
    print("La velocidad media de los ciclistas en el año {} es: {} km/h".format(año,combine["speed"].mean()))

In [426]:
interact(velocidad_año, año=[2014,2015,2016,2017])

interactive(children=(Dropdown(description='año', options=(2014, 2015, 2016, 2017), value=2014), Output()), _d…

<function __main__.velocidad_año>

### Cantidad de bicicletas totales para un momento dado. Considerando la misma como la cantidad de bicicletas que hay en todas las estaciones activas para ese momento, más todos los viajes que se estén realizando.

In [61]:
from datetime import datetime
import pytz

datetime_picker = ipydatetime.DatetimePicker()

def total_bikes_at_date(date):
    if date!=None:
        df = pd.read_csv("Data/OD_{}.csv".format(date.year))
        df.start_date = pd.to_datetime(df.start_date)
        df.end_date = pd.to_datetime(df.end_date)
        date = pd.to_datetime(date).tz_localize(tz=None)

        total = len(df[(date>df.start_date) & (date<df.end_date)])
        print("En el momento {} hay {} bicicletas".format(str(date),total))        
    else: 
        return 0
    
interact(total_bikes_at_date,date=datetime_picker)

interactive(children=(DatetimePicker(value=None, description='date'), Output()), _dom_classes=('widget-interac…

<function __main__.total_bikes_at_date>